In [291]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [292]:
train_clean = pd.read_csv('hw4-trainingset-wsa2113.csv')
train = train_clean.copy()

/Users/barmfield/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (50,53,54,55,56,255,256,257,258,260,268,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [293]:
test_clean = pd.read_csv('hw4-testset-wsa2113.csv')
test = test_clean.copy()

/Users/barmfield/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (50,255,256,257,258,260,268,280) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [294]:
cb = pd.read_csv('CodeBook-SELECT.csv')

In [295]:
print('train shape:',train.shape)
print('test shape:',test.shape)
print('code book shape:',cb.shape)

train shape: (20000, 380)
test shape: (24500, 380)
code book shape: (379, 2)


## TODO:  
* create baseline
    * drop high null variables and rows
* Data cleaning
    * drop duplicate columns
        * cntryid, cntryid_e
        * drop cntryid, as it is less specific
    * drop cols w/ high null %
    * drop rows w/ high null %
    * add is_null column for each category where value was null
    * find cols with mixed types and correct
    * convert numerical columns to int or float from objects
    * add label for categorical and numeric columns to codebook.csv, then make lists of the VarName for the feature transformers
    * categorical variables
        * OHC categorical variables
        * make sure dtypes match up across train and test
    * missing data
        * experiment imputing with mean and median
            * check distribution of data; if there are big outliers, use median, if not, use mean
        * check all cols which have missing data and see if that data should be captured in a new feature
    * feature scaling
        * normalization --> min/max scaler
        * standardization (z-score normalization)
        * api
            * column transformer
                * numeric transformer
                    * imputer
                    * scaler
                * categorical transformer
                    * imputer
                    * OHE
    * dimensionality reduction
        * PCA

# Baseline
* Remove columns
    * Drop high % null cols and rows
        * Threshold = 75%
    * Drop columns unrelated to performance
    * Drop target


In [296]:
train = train\
    .dropna(thresh = len(train)*0.25, axis = 1)\
    .dropna(thresh = len(train.columns)*0.25, axis = 0)

train_clean = train.drop(['uni','row','job_performance'], axis = 1)
test_clean = test[train_clean.columns]
print(train_clean.shape)
print(test_clean.shape)

(19994, 273)
(24500, 273)


Dropped 6 rows and 105 columns.

* Identify categorical and numerical columns

In [442]:
cat_cols = ['cntryid', 'cntryid_e','gender_r', 'computerexperience', 'nativespeaker',
            'edlevel3','monthlyincpr', 'yearlyincpr','lng_home','cnt_h','cnt_brth',
            'reg_tl2','lng_bq','lng_ci','vet','ctryqual','birthrgn','nativelang','ctryrgn',
            'imyrcat','ageg5lfs','ageg10lfs','ageg10lfs_t','edcat8','edcat6','fe12','aetpop',
            'faet12','faet12jr','faet12njr','nfe12','fnfaet12','edwork','neet','nopaidworkever',
            'paidwork12','iscoskil4','isic1l','nfe12jr','fnfe12jr','fnfaet12jr','fnfaet12njr',
            'paidwork5','earnhrdcl','earnhrbonusdcl','earnmthalldcl','earnflag','learnatwork_wle_ca',
            'readytolearn_wle_ca','icthome_wle_ca','ictwork_wle_ca','influence_wle_ca',
            'planning_wle_ca','readwork_wle_ca','taskdisc_wle_ca','writhome_wle_ca','writwork_wle_ca',
            'v59','v200','v31','v151','v272','v90','v157','v74','v153','v102', 'v101', 'v60', 'v285',
            'v17', 'v94', 'v137', 'v234', 'v91', 'v269', 'v236', 'v47', 'v291',
            'v227', 'v225', 'v201', 'v36', 'v209', 'v185', 'v238', 'v35',
            'v183', 'v5', 'v273', 'v259', 'v286', 'v95', 'v67', 'v217', 'v174',
            'v205', 'v46', 'v122', 'v92', 'v88', 'v179', 'v52', 'v97', 'v84',
            'v270', 'v33', 'v242', 'v140', 'v53', 'v82', 'v70', 'v184', 'v19',
            'v104', 'v48', 'v8', 'v115', 'v251', 'v3', 'v135', 'v235', 'v1',
            'v261', 'v263', 'v158', 'v244', 'v198', 'v212', 'v62', 'v221',
            'v214', 'v276', 'v246', 'v181', 'v96', 'v63', 'v87', 'v255',
            'v289', 'v77', 'v123', 'v141', 'v24', 'v193', 'v275', 'v204',
            'v108', 'v164', 'v166', 'v197', 'v34', 'v42', 'v292', 'v131',
            'v142', 'v188', 'v139', 'v247', 'v99', 'v180', 'v124', 'v51',
            'v190', 'v248', 'v229', 'v189', 'v165', 'v173', 'v134', 'v2',
            'v25', 'v18', 'v216', 'v178', 'v282', 'v13', 'v233', 'v278',
            'v103', 'v155', 'v152', 'v258', 'v277', 'v40', 'v146', 'v195',
            'v73', 'v23', 'v106', 'v271', 'v250', 'v176', 'v111', 'v218',
            'v253', 'v132', 'v284', 'v267', 'v260', 'v26', 'v171', 'v14', 'v7',
            'v240', 'v186', 'v162', 'v149', 'v228', 'v28', 'v237', 'v280',
            'v175', 'v288', 'v15', 'v208', 'v43', 'v27', 'v114', 'v191',
            'v170', 'v65', 'v57', 'v177', 'v69', 'v85', 'v50', 'v239', 'v224',
            'v71', 'v105','isic2l','isic1c','isic2c','isco1c','isco2c','isco1l',
            'isco2l','nfe12njr','readhome_wle_ca','v256'
           ]

num_cols = ['age_r','yrsqual','yrsqual_t','yrsget','edcat7','leavedu','nfehrsnjr','nfehrsjr',
            'nfehrs','earnhr','earnhrppp','earnhrbonus','earnhrbonusppp','earnmth',
            'earnmthppp','earnmthbonus','earnmthall','earnmthallppp','earnmthbonusppp','learnatwork',
            'readytolearn','icthome','ictwork','influence','planning','readhome','readwork','taskdisc',
            'writhome','writwork','v202','v231'
           ]
num_rows = train_clean.shape[0]

In [443]:
l = list()
seen = list()
for lis in [cat_cols, num_cols]:
    for col in lis:
        l.append(col)
for c in sorted(l):
    if c in seen:
        print(c)
        break
    else:
        seen.append(c)
#     if c not in train_clean.columns.values:
#         print(c)

In [444]:
len(cat_cols) + len(num_cols)

273

* Fix columns with multiple data types
    * Impute missing values
    * Find columns with mixed types
    * Set column types

In [441]:
show_info('isic2l')

9996      11398
9995       3134
9996.0     1339
9999        328
47          324
9995.0      321
85          224
56          185
84          114
86          104
43           91
41           75
01           75
10           70
88           66
49           60
81           55
46           52
96           43
55           42
9999.0       40
87           38
45           35
64           34
47.0         33
93           33
28           30
82           29
85.0         27
29           26
          ...  
9997          2
68.0          2
66.0          2
11.0          1
06            1
15.0          1
17.0          1
18.0          1
20.0          1
H             1
09            1
99            1
23.0          1
16.0          1
63.0          1
95.0          1
61.0          1
02            1
33.0          1
38.0          1
82.0          1
42.0          1
79.0          1
74.0          1
72.0          1
07            1
69.0          1
65.0          1
60.0          1
70.0          1
Name: isic2l, Length: 15

In [300]:
print(train_clean.shape)
print(len(train_clean.columns))
clean_cols = train_clean.columns.values

(19994, 273)
273


In [274]:
len(clean_cols)

273

In [277]:
col_imputer = ColumnTransformer(
    [
        ('cat', SimpleImputer(strategy='constant', fill_value='missing'), cat_cols),
        ('num', SimpleImputer(strategy='median'), num_cols)
    ]
)

col_imputer.fit_transform(train_clean).shape

(19994, 273)

In [301]:
# impute

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

col_imputer = ColumnTransformer(
    [
        ('cat', SimpleImputer(strategy='constant', fill_value='missing'), cat_cols),
        ('num', SimpleImputer(strategy='median'), num_cols)
    ]
)

clean_cols = train_clean.columns.values
train_imp = pd.DataFrame(data = col_imputer.fit_transform(train_clean), columns = clean_cols)

In [304]:
train_imp.head()

,cntryid,cntryid_e,age_r,gender_r,computerexperience,nativespeaker,edlevel3,monthlyincpr,yearlyincpr,lng_home,...,v65,v57,v177,v69,v85,v50,v239,v224,v71,v105
0,France,France,44.0,Male,Yes,Yes,Medium,25 to less than 50,50 to less than 75,fra,...,Strongly agree,Strongly agree,Strongly agree,3114,9996,2410,9996,14.6238,1.77183,2.27432
1,United States,United States,missing,Male,Yes,Yes,High,50 to less than 75,50 to less than 75,999,...,Strongly disagree,Strongly disagree,Neither agree nor disagree,9999,9996,9999,9996,12.2562,2.4697,2.24364
2,Korea,Korea,17.0,Female,Yes,Yes,Low,missing,missing,kor,...,Disagree,Agree,Agree,9996,9996,9996,9996,12.2562,2.50904,2.04642
3,France,France,44.0,Female,No,No,Medium,10 to less than 25,10 to less than 25,fra,...,Strongly agree,Strongly agree,Strongly agree,9112,9996,8121,9996,12.1223,0.918549,0.572892
4,Japan,Japan,64.0,Male,Yes,Yes,High,Less than 10,Less than 10,jpn,...,Strongly agree,Agree,Neither agree nor disagree,3332,9996,8220,9996,9.01823,0.962018,1.67983


In [305]:
# find columns with mixed types and normalize types

for col in train_imp.columns:
    print(col)
    if col in cat_cols:
        train_imp[col] = train_imp[col].astype(str)
    else:
        train_imp[col] = train_imp[col].astype(float)


cntryid
cntryid_e
age_r
gender_r
computerexperience
nativespeaker
edlevel3
monthlyincpr
yearlyincpr
lng_home
cnt_h
cnt_brth
reg_tl2
lng_bq
lng_ci
yrsqual
yrsqual_t
yrsget
vet
ctryqual
birthrgn
nativelang
ctryrgn
imyrcat
ageg5lfs
ageg10lfs
ageg10lfs_t
edcat8
edcat7
edcat6
leavedu
fe12
aetpop
faet12
faet12jr
faet12njr
nfe12
fnfaet12
edwork
neet
nfehrsnjr
nfehrsjr
nfehrs
nopaidworkever
paidwork12
iscoskil4
isic1l
isic2l
isic1c
isic2c
isco1c
isco2c
isco1l
isco2l
earnhr
earnhrppp
earnhrbonus
earnhrbonusppp


ValueError: could not convert string to float: 'missing'

* Impute missing values

In [180]:
for col in mixed_cols:
    t = type(train_clean[col].iloc[0])
    print(col)
    train_clean[col] = train_clean[col].astype(t)

In [237]:
# train_clean['earnhrbonusppp']
'earnhrbonusppp' in num_cols


True

In [246]:
# first = train_clean['earnhrbonusppp']
first

0        14.623844
1              NaN
2              NaN
3        12.122307
4         9.018234
5         9.475108
6        17.005075
7        11.588129
8              NaN
9         5.899280
10       13.012549
11        6.905825
12             NaN
13        5.248767
14       26.214236
15             NaN
16        7.505750
17             NaN
18       11.922821
19             NaN
20             NaN
21             NaN
22             NaN
23       11.805689
24             NaN
25             NaN
26             NaN
27             NaN
28             NaN
29             NaN
           ...    
19970     9.735364
19971          NaN
19972    18.180760
19973     5.109512
19974          NaN
19975    19.375111
19976     5.586357
19977          NaN
19978    44.243635
19979          NaN
19980          NaN
19981          NaN
19982          NaN
19983          NaN
19984          NaN
19985     4.116680
19986    22.502229
19987    28.361305
19988     7.305483
19989          NaN
19990          NaN
19991       

In [245]:
train_clean['earnhrbonusppp']

0           1847
1        missing
2        missing
3        1832.09
4          50000
5        1.6e+06
6           1807
7           1500
8        missing
9            762
10           400
11         26000
12       missing
13          1700
14          2288
15       missing
16       1.6e+06
17       missing
18         2e+06
19       missing
20       missing
21       missing
22       missing
23        300000
24       missing
25       missing
26       missing
27       missing
28       missing
29       missing
          ...   
19964    90133.3
19965    missing
19966     420000
19967      16000
19968    missing
19969       3000
19970       1800
19971    missing
19972    5416.67
19973    missing
19974    missing
19975    missing
19976    missing
19977    missing
19978    missing
19979       1500
19980     500000
19981       3750
19982       1000
19983    missing
19984    missing
19985    missing
19986    missing
19987       1380
19988    missing
19989    21666.7
19990    missing
19991       27

### Build pipeline
    * impute nulls
    * scale numerical columns
    * OHE categorical columns
    * Lasso regression to remove variables

In [171]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV

num_tranform = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

cat_transform = Pipeline(
    [
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

preprocessor = ColumnTransformer(
    [
        ('num', num_tranform, num_cols),
        ('cat', cat_transform, cat_cols)
    ]
)

clf = Pipeline(
    [
        ('preprocessor', preprocessor),
        ('classifier', LinearRegression())
    ]
)

X_train, X_test, y_train, y_test = train_test_split(X_train_clean, y_train_clean, random_state = 4771)

clf.fit(X_train, y_train)
              

TypeError: '<' not supported between instances of 'str' and 'float'

In [160]:
train_test_split?

Signature: train_test_split(*arrays, **options)
Docstring:
Split arrays or matrices into random train and test subsets

Quick utility that wraps input validation and
``next(ShuffleSplit().split(X, y))`` and application to input data
into a single call for splitting (and optionally subsampling) data in a
oneliner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float, int or None, optional (default=0.25)
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. By default, the value is set to 0.25.
    The default will change in version 0.21. It will remain 0.25 only
    if ``train_size`` is unspecified, o

In [105]:
# find columns with mixed types

# for col in train.columns:
#     weird = (train[[col]].applymap(type) != train[[col]].iloc[0].apply(type)).any(axis=1)
#     if len(train[weird]) > 0:
#         print(col)

In [71]:
train.dtypes

Index(['isic2l', 'isco1c', 'isco2c', 'isco1l', 'isco2l', 'v76', 'v144', 'v199',
       'v44', 'v109', 'v172', 'v71', 'v10'],
      dtype='object')

In [58]:
train[train.cntryid != train.cntryid_e]

In [59]:
# train.isnull().sum().sort_values(ascending=False)/train.shape[0]> .5

In [77]:
train.shape

(19994, 275)

Drop columns greater than 75% null